In [ ]:
# default_exp core

# Core

> API details.

In [4]:
#hide
from nbdev.showdoc import *

In [10]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code_test.ipynb.


In [6]:
#export
import IPython.core
import json
import os.path
import re
import ipykernel
import requests

# Alternative that works for both Python 2 and 3:
from requests.compat import urljoin

try:  # Python 3 (see Edit2 below for why this may not work in Python 2)
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_path():
    """
    Return the full path of the jupyter notebook.
    !!! Doesn't work from VS code.
    """
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        try:
            response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                    params={'token': ss.get('token', '')}, timeout=1)
            for nn in json.loads(response.text):
                if nn['kernel']['id'] == kernel_id:
                    relative_path = nn['notebook']['path']
                    return os.path.join(ss['notebook_dir'], relative_path)
        except Exception as e:
            pass


In [4]:
#hide
get_notebook_path()

'/Users/alekseyv/vlasenkoalexey/gcp_runner/00_core.ipynb'

In [5]:
#export
import nbdev.export
import nbdev.imports
import nbformat

from gcp_runner.core import get_notebook_path

if nbdev.imports.in_colab():
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

def read_colab_nb(notebook_path):
    gdrive_file_id = re.search('/fileId=(.+)', notebook_path)[1]
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    gdrive = GoogleDrive(gauth)
    downloaded = gdrive.CreateFile({'id': gdrive_file_id})
    downloaded_string = downloaded.GetContentString()
    return nbformat.reads(downloaded_string, as_version=4)

def find_default_export_for_notebook(notebook_path):
    nb = read_colab_nb(notebook_path) \
        if nbdev.imports.in_colab() else nbdev.export.read_nb(notebook_path)
    default = nbdev.export.find_default_export(nb['cells'])
    return default

def get_module_name():
    return find_default_export_for_notebook(get_notebook_path())

In [6]:
#hide
get_module_name()

'core'

In [1]:
#export

import pkgutil
import importlib
import time
import nbdev.imports
from nbdev.export import *

def get_package_name():
    "Returns name of the current package"
    # see logic in https://github.com/fastai/nbdev/blob/master/nbdev/export.py
    return str(nbdev.imports.Config().lib_path).split('/')[-1]

def reload_package_modules(package_name):
    "Dynamically reloads package `package_name`."
    # Reload order matters. First we need to reload all modules except 
    # the current one, and then reload current one. Otherwise reload might
    # fail if current module refers to other module that has new functions added.
    current_module_name = get_module_name()
    current_module = None
    for _, module_name, _ in (pkgutil.iter_modules([package_name])):
        if current_module_name == module_name:
            current_module
        else:
            module = importlib.import_module(package_name + '.' + module_name)
            importlib.reload(module)
    if current_module is not None:
        importlib.reload(current_module)
        
def export_and_reload_all(silent=False, ignore_errors=False):
    "Converts all notebooks to modules, and reloads all modules"
    if nbdev.imports.in_ipython():
        notebook2script(silent=silent)
        # Need to reload packages in order
        # Looks like when it is called with to_dict, it doesn't actually update files.
        module_name_dict = notebook2script(silent=True, to_dict=True)
        time.sleep(1)
        #reload_package_modules(get_package_name())
        module_name_to_notebook_path = \
            list ((key,str(value[0][1])) for key, value in module_name_dict.items())
        sorted_list = sorted(module_name_to_notebook_path, key=lambda pair:pair[1])
        package_name = get_package_name()
        for (module_name, _) in sorted_list:
            if ignore_errors:
                try:
                    module = importlib.import_module(package_name + '.' + module_name)
                    importlib.reload(module)
                except Error as e:
                    print (e)
            else:
                module = importlib.import_module(package_name + '.' + module_name)
                importlib.reload(module)


In [4]:
export_and_reload_all()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code_test.ipynb.


In [9]:
#export

import os
import subprocess
import io

from threading import Thread
from queue import Queue 

def _reader(pipe, queue, pipe_name):
    try:
        with pipe:
            for line in iter(pipe.readline, ''):
                queue.put((pipe_name, line))
    finally:
        queue.put(None)
        
def run_process(args, **kwargs):
    proc = subprocess.Popen(
        args, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE, 
        bufsize=1,
        encoding='utf-8')
    
    q = Queue()
    Thread(target=_reader, args=[proc.stdout, q, 'stdout']).start()
    Thread(target=_reader, args=[proc.stderr, q, 'stderr']).start()
    for source, line in iter(q.get, None):
        line = line.rstrip('\n').rstrip('\r')
        if line != '':
            if source == 'stderr':
                print("\x1b[31m{}\x1b[0m".format(line))
            else:
                print(line)
    
    return proc.poll()

In [10]:
#export

def get_run_args(func, **kwargs):
    package_name = get_package_name()
    module_name = get_module_name()
    function_name = func.__name__
    args = ['--module-name=%s' % package_name + '.' + module_name,
         '--function-name=%s' % function_name]
    if kwargs is not None:
        for key,value in kwargs.items():
            args.append('--%s=%s' % (key.replace('_', '-'), str(value)))
    
    return args

def get_run_python_args(func, python_binary='python', **kwargs):
    args = [python_binary, 
         '-u',
         '-m',
         'gcp_runner.entry_point']
    
    args.extend(get_run_args(func, **kwargs))
    return args

In [11]:
get_run_args(get_run_args)

['--module-name=gcp_runner.core', '--function-name=get_run_args']

In [3]:
export_and_reload_all()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code_test.ipynb.


In [ ]:
#export
def build_and_push_docker_image(docker_file_path, image_uri, push_docker=True, dry_run=False):
    build_docker_args = ['docker', 'build', '-f', docker_file_path, '-t', image_uri, './']
    print('Building Docker image:')
    print(' '.join(build_docker_args))
    if not dry_run:
        result = run_process(build_docker_args)
        if result:
            return result
    if push_docker:
        push_docker_args = ['docker', 'push', image_uri]
        print('Pushing Docker image:')
        print(' '.join(push_docker_args))
        if not dry_run:
            result = run_process(push_docker_args)
            if result:
                return result
    return 0

In [28]:
#export
import datetime
import getpass

def format_job_dir(job_dir, date_time:datetime.datetime=None):
    if date_time is None:
        date_time = datetime.datetime.now()
    return job_dir.format(**{'datetime':datetime.datetime.now().strftime('%Y%m%d_%H%M%S'), 'username': getpass.getuser()})

format_job_dir('model_{datetime}_{username}')

'model_20200409_154904_alekseyv'

In [29]:
#export

def print_tensorboard_command(job_dir):
    print('To see job output in tensorboard, run following command:')
    print('tensorboard --logdir=%s' % job_dir)
    

In [31]:

import os

def setup_service_account(user_name=None, project_name=None, service_account_key_file_path='./service_account.json'):
    if project_name is None:
        project_name = run_process('gcloud config get-value project --quiet'.split(' '))
        if project_name is None or project_name == '':
            print('gcloud project is not configured, please set it up first by running:'
                 ' gcloud config set project <PROJECT_NAME>')
            return -1
    if user_name is None:
        if nbdev.imports.in_colab():
            from google.colab import auth
            auth.authenticate_user()
            user_name = run_process('gcloud config get-value account --quiet'.split(' '))
            if user_name is None or user_name == '':
                print("can't identify user name, please specify it explicitly")
                return -1
            if '@' in user_name:
                user_name = user_name[:user_name.index('@')]
        else:
            user_name = os.environ['USER']
            if user_name is None or user_name == '':
                print("can't identify user name, please specify it explicitly")
                return -1
    
setup_service_account()    

alekseyv-scalableai-dev


In [35]:
import os
os.environ['USER']

'alekseyv'

In [23]:
! gcloud config get-value project --quiet

alekseyv-scalableai-dev
